<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/VAE_PSA_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import datetime
import matplotlib.pyplot as plt
#from IPython import display
import numpy as np
import glob
import cv2
import pandas as pd
# set a random seed
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
AUTO = tf.data.experimental.AUTOTUNE
# parameters for building the model and training

#CV: Width , Height
#Tensor: Height, Width
"""
Defining Functions
"""
COL = 256 #width
ROW = 128 #height
INPUT_SHAPE=(ROW, COL, 1,)
BATCH_SIZE = 128
LATENT_DIM = 128
def get_dataset(image_dir):
    image_file_list = os.listdir(image_dir)

    image_paths = [os.path.join(image_dir, fname) for fname in image_file_list]
    random.shuffle(image_paths)
    train_data = tf.data.Dataset.from_tensor_slices((image_paths))
    print("Training dataset: {} images".format(len(image_paths)))
    return train_data, len(image_paths)


def pre_image(image_filename):
    img_raw = tf.io.read_file(image_filename)
    #img_raw = tf.io.read_file(r'E:\1.Project\6.PSA\Left\Images\Label\A3DV1S42ANQBC09620240531112039_L.bmp')
    #print('read file :', image_filename)
    #image = tf.image.decode_image(img_raw)
    #image = tf.cast(image, dtype=tf.float32)

    #image = tf.image.resize(image, (COL, ROW))
    image = tf.image.decode_image(img_raw, expand_animations=False)
    image = tf.cast(image, dtype=tf.float32)

    image = tf.image.resize(image, [ROW, COL], method='nearest')
    image = image / 255.0
    image = tf.reshape(image, shape=(ROW, COL, 1,))
    print(image.shape)
    # cv2.imshow('',image.numpy())
    return image

class Sampling(layers.Layer):
    def call(self, inputs):
        mu, sigma = inputs
        batch = tf.shape(mu)[0]
        dim = tf.shape(mu)[1]
        epsilon = keras.backend.random_normal(shape=(batch, dim))
        z = mu + tf.exp(0.5 * sigma) * epsilon
        # z = mu + tf.exp(0.5 * sigma)
        return z
def encoder_conv_block(inputs,filter):
    e1 = layers.Conv2D(filters=2**filter, kernel_size=3, strides=(1,1), padding="same",dilation_rate=(1,1), activation='relu')(inputs)
    e1 = layers.BatchNormalization()(e1)

    e2 = layers.Conv2D(filters=2**filter+1, kernel_size=3, strides=(1,1), padding="same", dilation_rate=(2, 2), activation='relu')(inputs)
    e2 = layers.BatchNormalization()(e2)

    e = layers.concatenate(inputs=[e1, e2], axis=-1)
    y = layers.Conv2D(filters=2**filter+1, kernel_size=3, strides=(2,2), padding="same", dilation_rate=(1, 1), activation='relu')(e)
    return y

# def encoder_layers(inputs, latent_dim):
#     x = layers.Conv2D(filters=4, kernel_size=3, strides=2, padding="same", activation='relu')(inputs)
#     x = layers.BatchNormalization()(x)
#
#     #batuan
#     # x = layers.Conv2D(filters=8, kernel_size=3, strides=2, padding='same', activation='relu')(x)
#     # x = layers.BatchNormalization()(x)
#     # batuan
#
#     x = layers.Conv2D(filters=16, kernel_size=3, strides=2, padding='same', activation='relu')(x)
#     x = layers.BatchNormalization()(x)
#     # x = layers.Flatten()(x)
#
#     x = layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu')(x)
#     batch_3 = layers.BatchNormalization()(x)
#
#     x = layers.Flatten()(batch_3)
#
#     mu = layers.Dense(latent_dim, name='latent_mu')(x)
#     sigma = layers.Dense(latent_dim, name='latent_sigma')(x)
#     return mu, sigma, batch_3.shape

# def encoder_layers(inputs, latent_dim):
#     x = encoder_conv_block(inputs=inputs,filter=6)
#     x = layers.BatchNormalization()(x)
#
#     # x = layers.Conv2D(filters=8, kernel_size=3, strides=2, padding='same', activation='relu')(x)
#     # x = layers.BatchNormalization()(x)
#
#     y = encoder_conv_block(inputs=x,filter=7)
#     z = layers.concatenate([x, y], name="sum",axis=1 )
#     z = layers.BatchNormalization()(z)
#
#     z = encoder_conv_block(inputs=x,filter=8)
#     batch_3 = layers.BatchNormalization()(z)
#     z = layers.Flatten()(batch_3)
#     mu = layers.Dense(latent_dim, name='latent_mu')(z)
#     sigma = layers.Dense(latent_dim, name='latent_sigma')(z)
#     return mu, sigma, batch_3.shape
def encoder_layers(inputs, latent_dim):
    x = encoder_conv_block(inputs=inputs,filter=3)
    x = layers.BatchNormalization()(x)

    # x = layers.Conv2D(filters=8, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    # x = layers.BatchNormalization()(x)

    x = encoder_conv_block(inputs=x,filter=4)
    x = layers.BatchNormalization()(x)

    x = encoder_conv_block(inputs=x,filter=5)
    batch_3 = layers.BatchNormalization()(x)

    x = layers.Flatten()(batch_3)

    mu = layers.Dense(latent_dim, name='latent_mu')(x)
    sigma = layers.Dense(latent_dim, name='latent_sigma')(x)
    return mu, sigma, batch_3.shape


def encoder_model(latent_dim, input_shape):
    inputs = layers.Input(shape=input_shape)
    mu, sigma, conv_shape = encoder_layers(inputs, latent_dim=latent_dim)
    z = Sampling()((mu, sigma))
    model = keras.Model(inputs, outputs=[mu, sigma, z], name='Encoder')
    model.summary()
    # keras.utils.plot_model(
    #     model,
    #     to_file='encoder.png',
    #     show_shapes=True,
    #     show_layer_names=True
    # )
    return model, conv_shape

def decoder_layers(inputs, conv_shape):
    units = conv_shape[1] * conv_shape[2] * conv_shape[3]
    x = layers.Dense(units, activation='relu')(inputs)
    x = layers.BatchNormalization()(x)

    x = layers.Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)

    x = layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(filters=16, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # x = layers.Conv2DTranspose(filters=8, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    # x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(filters=4, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same', activation='sigmoid')(x)
    return x
def decoder_model(latent_dim, conv_shape):
    inputs = layers.Input(shape=(latent_dim,))
    outputs = decoder_layers(inputs, conv_shape)
    model = keras.Model(inputs, outputs, name='Decoder')
    model.summary()
    # keras.utils.plot_model(
    #     model,
    #     to_file='decoder.png',
    #     show_shapes=True,
    #     show_layer_names=True
    # )
    return model
def kl_reconstruction_loss(inputs, outputs, mu, sigma):
    kl_loss = 1 + sigma - tf.square(mu) - tf.math.exp

def kl_reconstruction_loss(inputs, outputs, mu, sigma):
    kl_loss = 1 + sigma - tf.square(mu) - tf.math.exp(sigma)
    return tf.reduce_mean(kl_loss) * -0.5
def vae_model(encoder, decoder, input_shape):
    inputs = keras.layers.Input(shape=input_shape)
    mu, sigma, z = encoder(inputs)
    reconstructed = decoder(z)
    model = keras.Model(inputs=inputs, outputs=reconstructed)
    loss = kl_reconstruction_loss(inputs, z, mu, sigma)
    model.add_loss(loss)
    return model

def get_vae_models(input_shape, latent_dim):
    encoder, conv_shape = encoder_model(latent_dim=latent_dim, input_shape=input_shape)
    decoder = decoder_model(latent_dim=latent_dim, conv_shape=conv_shape)
    vae = vae_model(encoder, decoder, input_shape=input_shape)
    return encoder, decoder, vae


def generate_and_save_images(model, epoch, step, test_input):
    predictions = model.predict(test_input)
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        img = predictions[i, :, :, 0] * 255
        img = img.astype('int32')
        plt.imshow(img, cmap='gray')
        plt.axis('off')

    fig.suptitle("epoch: {}, step: {}".format(epoch, step))
    plt.savefig('image_at_epoch_{:04d}_step{:04d}.png'.format(epoch, step))
    fig.clear()
    plt.close(fig)
    # plt.show()


"""
VAE model
"""

def Train_VAE(datapath="", epochs=30000, use_transferlearning=False, model_path=""):
    if use_transferlearning:
        vae = tf.keras.models.load_model(model_path)

    else:
        encoder, decoder, vae = get_vae_models(input_shape=INPUT_SHAPE, latent_dim=LATENT_DIM)

    optimizer = keras.optimizers.Adam(learning_rate=0.0005)
    loss_metric = keras.metrics.Mean()
    mse_loss = keras.losses.MeanSquaredError()

    '''
    Preparing dataset
    '''
    train_data, no_train = get_dataset(datapath)
    train_ds = (train_data
                .shuffle(no_train)
                .map(pre_image, num_parallel_calls=AUTO)
                .batch(BATCH_SIZE)
                .prefetch(buffer_size=AUTO))

    '''
    Training loop
    '''
    # random_vector_for_generation = tf.random.normal(shape=[16, LATENT_DIM])
    # generate_and_save_images(decoder, 0, 0, random_vector_for_generation)

    for epoch in range(epochs):
        print('Start of epoch %d' % (epoch,))
        for step, x_batch_train in enumerate(train_ds):
            print(f"SHAPE= {x_batch_train.shape}")
            with tf.GradientTape() as tape:
                reconstructed = vae(x_batch_train)
                flattened_inputs = tf.reshape(x_batch_train, shape=[-1])
                flattened_outputs = tf.reshape(reconstructed, shape=[-1])

                loss = mse_loss(flattened_inputs, flattened_outputs) * COL * ROW
                loss += sum(vae.losses)

            grads = tape.gradient(loss, vae.trainable_weights)
            optimizer.apply_gradients(zip(grads, vae.trainable_weights))

            loss_metric(loss)
            print('Epoch: %s step: %s mean loss = %s' % (epoch, step, loss_metric.result().numpy()))
        if epoch % 100 == 0:
            vae.save(os.path.join(model_path,f'PSA_Vae_TAB1_L4__{epoch}__{loss}'),
                     save_format="tf")

# datapath = r'E:\ToanPT\1.Code_train_Unet\data\PSA\DATA-TRAIN\LINE 4\TRAIN\TAB6'
# Train_VAE(datapath=datapath,epochs=10001,use_transferlearning = False,
#           model_path=r"E:\VideoClassification\model\PSA\LINE4\VAE\VAE-TAB6-NEW-L4")


print("DONE")

#Test model
def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])

    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err


model_path=r'E:\VideoClassification\model\PSA\LINE4\VAE\VAE-TAB1-NEW-L4\PSA_Vae_TAB1_L4__10000__3.9850244522094727'
vae = tf.keras.models.load_model(model_path)

def Predict(file):
    #vae = tf.keras.models.load_model(model_path)
    img = cv2.imread(file,0)
    img = cv2.resize(img, (COL, ROW))
    x = img.astype('float32') / 255.0
    vae_img = x.reshape((1, ROW, COL, 1))
    x_recon = vae.predict(vae_img)
    x_recon=x_recon.reshape(ROW,COL)
    x_input = vae_img.reshape(ROW, COL)
    mse_error = mse(x_input, x_recon)
    return mse_error

def Eval():
    NG=0
    OK=0
    dir_predict=r'E:\ToanPT\1.Code_train_Unet\data\PSA\DATA-TRAIN\LINE 4\NG\LEFT\TAB1'
    image_file_list = os.listdir(dir_predict)
    image_paths = [os.path.join(dir_predict, fname) for fname in image_file_list]
    n = len(image_paths)
    R = np.zeros(n)
    print(f"Number Images ={n}")
    for i in range(n):
        e1 = Predict(image_paths[i])
        if e1 < 0.00018:
            OK =OK +1
        elif e1 > 0.002:
            NG= NG+1
        R[i] = e1

    print(f"MAX= {np.max(R)}")
    print(f"MIN= {np.min(R)}")
    print(f"STD= {np.std(R)}")
    print(f"AVG={np.average(R)}")
    print(f"OK={OK}, NG={NG}")

    # df = pd.DataFrame({"Value": R})
    # df.to_excel(r"D:\Private_Documents\VAE-PSA-TAB1.xlsx", index=False)

    # e1 = Predict(model_path, image_paths[0])
    # print(e1)

Eval()

# mse = np.mean(np.power(x_test - x_test_decoded,2)m axis=1)
# threshold = np.max(mse)